In [1]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt 
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [11]:
def copy_before(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) == 0:
        yt.copy("//home/cloud_analytics/emailing/sender_copy/" + table_name,
                "//home/cloud_analytics/emailing/sender/" + table_name,
                force=True)
        print('saved')
copy_before("ba-activation")

In [15]:
df = lib.execute_query("""
SELECT
    t0.*,
    language
FROM(
    SELECT
        multiIf(
            user_settings_email LIKE '%@yandex.%' OR user_settings_email LIKE '%@ya.%',CONCAT(lower(replaceAll(splitByString('@',assumeNotNull(user_settings_email))[1], '.', '-')), '@yandex.ru'),
            lower(user_settings_email)
        ) as email,
        toUInt64(puid) as puid,
        cloud_id,
        event_time as console_regstration_date,
        addDays(toDate(console_regstration_date), 2) as experiment_date,
        multiIf(first_ba_created_datetime = '0000-00-00 00:00:00', '', first_ba_created_datetime) as ba_created,
        mail_feature,
        mail_info,
        mail_promo,
        toDate(NOW()) - toDate(console_regstration_date) as days_sinse_console_registration,
        modulo(puid, 100) as group_key,
        multiIf(
            modulo(puid, 100) < 35, 'test',
            modulo(puid, 100) > 44, 'test',
            'control'
        ) as Group
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
    WHERE
        event = 'cloud_created'
        AND toDate(event_time) >= toDate('2020-06-01')
) as t0
ANY LEFT JOIN(
    SELECT
        cloud_id,
        user_settings_language as language
    FROM "//home/cloud_analytics/import/iam/cloud_owners_history"
) as t1
ON t0.cloud_id = t1.cloud_id
FORMAT TabSeparatedWithNames
""")

In [16]:
lib.save_table("ba-activation", "//home/cloud_analytics/emailing/sender", df)

In [14]:
def copy_after(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) != 0:
        yt.copy("//home/cloud_analytics/emailing/sender/" + table_name,
                "//home/cloud_analytics/emailing/sender_copy/" + table_name, force=True)
copy_after("ba-activation")